In [75]:
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np

In [3]:
url = 'https://www.imdb.com/list/ls021051704/?page=1'
eps = get(url)
soup = BeautifulSoup(eps.text, 'html.parser')

In [28]:
boxes = soup.find_all('div', class_="lister-item mode-detail")
time = boxes[1].find('span',class_="runtime").get_text().replace(' min', '')
time = int(time)
time

22

In [29]:
genre1 = boxes[1].find('span',class_="genre").get_text().strip().split(', ')[0]
genre2 = boxes[1].find('span',class_="genre").get_text().strip().split(', ')[1]
genre1

'Comedy'

In [30]:
star = float(boxes[1].find('span', class_="ipl-rating-star__rating").get_text())
star

8.3

In [32]:
summary = boxes[1].find('p',class_="").get_text().strip()
summary

'Monica and the gang introduce Rachel to the "real world" after she leaves her fiancé at the altar.'

In [39]:
director = boxes[1].find_all('p',class_="text-muted text-small")[1].a.text


'James Burrows'

In [48]:
title = boxes[1].find('h3').find_all('a')[1].text

'The One Where Monica Gets a Roommate'

In [53]:
votes = boxes[1].find('span',attrs={'name':"nv"}).get_text().replace(',', '')
votes = int(votes)
votes

6498

In [62]:
year = boxes[1].find_all('span', class_="lister-item-year text-muted unbold")[1].get_text()[1:-1]
year = int(year)
year

1994

In [70]:
rate = boxes[1].find('div',class_="list-description").get_text().replace('Rating: ', '')
rate = float(rate)
rate

9.2

In [95]:
years = []
votes = []
titles = []
dirs = []
sums = []
stars = []
genres1 = []
genres2 = []
times = []
rates = []

In [96]:
for i in range(1,4):
    url = 'https://www.imdb.com/list/ls021051704/?page='+str(i)
    eps = get(url)
    soup = BeautifulSoup(eps.text, 'html.parser')
    print(f'Starting page {i}')
    boxes = soup.find_all('div', class_="lister-item mode-detail")
    for j in range(len(boxes)):
        time = boxes[j].find('span',class_="runtime").get_text().replace(' min', '')
        time = int(time)
        times.append(time)
        
        genre1 = boxes[j].find('span',class_="genre").get_text().strip().split(', ')[0]
        genre2 = boxes[j].find('span',class_="genre").get_text().strip().split(', ')[1]
        genres1.append(genre1)
        genres2.append(genre2)

        star = float(boxes[j].find('span', class_="ipl-rating-star__rating").get_text())
        stars.append(star)
        
        summary = boxes[j].find('p',class_="").get_text().strip()
        sums.append(summary)
        
        director = boxes[j].find_all('p',class_="text-muted text-small")[1].a.text
        dirs.append(director)
        
        try:
            title = boxes[j].find('h3').find_all('a')[1].text
        except IndexError:
            title = boxes[j].find('h3').find_all('a')[0].text
        titles.append(title)
        
        vote = boxes[j].find('span',attrs={'name':"nv"}).get_text().replace(',', '')
        vote = int(vote)
        votes.append(vote)
        
        try:
            year = boxes[j].find_all('span',
                                 class_="lister-item-year text-muted unbold")[1].get_text()[1:-1]
            year = int(year)
        except IndexError:
            year = np.nan
        years.append(year)
        
        rate = boxes[j].find('div',class_="list-description").get_text().replace('Rating: ', '')
        try:
            rate = float(rate)
        except ValueError:
            try:
                rate = float(rate[:2])
            except ValueError:
                rate = np.nan
        rates.append(rate)
    print(f'Page {i} ended.')

Starting page 1
Page 1 ended.
Starting page 2
Page 2 ended.
Starting page 3
Page 3 ended.


In [109]:
df = pd.DataFrame({
    'Year': years,
    'Episode_Title': titles,
    'Duration': times,
    'Summary': sums,
    'Genre_1': genre1,
    'Genre_2': genre2,
    'Director': dirs,
    'Stars': stars,
    'Votes': votes,
    'Rating': rates
})
df.head(3)

,Year,Episode_Title,Duration,Summary,Genre_1,Genre_2,Director,Stars,Votes,Rating
0,NaN,Friends,22,Follows the personal and professional lives of...,Comedy,Romance,Jennifer Aniston,8.9,754303,NaN
1,1994.0,The One Where Monica Gets a Roommate,22,"Monica and the gang introduce Rachel to the ""r...",Comedy,Romance,James Burrows,8.3,6500,9.2
2,1994.0,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,Comedy,Romance,James Burrows,8.1,4888,8.4


In [121]:
df = df.drop('level_0',axis=1)
df = df.drop('index',axis=1)
df.head()

,Year,Episode_Title,Duration,Summary,Genre_1,Genre_2,Director,Stars,Votes,Rating
0,1994.0,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,Comedy,Romance,James Burrows,8.1,4888,8.4
1,1994.0,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,Comedy,Romance,James Burrows,8.2,4605,8.8
2,1994.0,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,Comedy,Romance,James Burrows,8.1,4468,8.7
3,1994.0,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Comedy,Romance,Pamela Fryman,8.5,4438,8.8
4,1994.0,The One with the Butt,22,Monica's obsessiveness is put to the test afte...,Comedy,Romance,Arlene Sanford,8.1,4274,8.3


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           235 non-null    float64
 1   Episode_Title  235 non-null    object 
 2   Duration       235 non-null    int64  
 3   Summary        235 non-null    object 
 4   Genre_1        235 non-null    object 
 5   Genre_2        235 non-null    object 
 6   Director       235 non-null    object 
 7   Stars          235 non-null    float64
 8   Votes          235 non-null    int64  
 9   Rating         235 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 13.8+ KB


In [124]:
df.Year = df.Year.astype('int')
df.head()

,Year,Episode_Title,Duration,Summary,Genre_1,Genre_2,Director,Stars,Votes,Rating
0,1994,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,Comedy,Romance,James Burrows,8.1,4888,8.4
1,1994,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,Comedy,Romance,James Burrows,8.2,4605,8.8
2,1994,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,Comedy,Romance,James Burrows,8.1,4468,8.7
3,1994,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Comedy,Romance,Pamela Fryman,8.5,4438,8.8
4,1994,The One with the Butt,22,Monica's obsessiveness is put to the test afte...,Comedy,Romance,Arlene Sanford,8.1,4274,8.3


In [135]:
df.drop(['Genre_1', 'Genre_2'], axis=1, inplace=True)
df.head()

,Year,Episode_Title,Duration,Summary,Director,Stars,Votes,Rating
0,1994,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.1,4888,8.4
1,1994,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.2,4605,8.8
2,1994,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.1,4468,8.7
3,1994,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.5,4438,8.8
4,1994,The One with the Butt,22,Monica's obsessiveness is put to the test afte...,Arlene Sanford,8.1,4274,8.3


In [136]:
df.to_csv('friends_episodes_v2.csv')